In [1]:
import pandas as pd
import numpy as np
import scipy.stats

gams_python_path="/Library/Frameworks/GAMS.framwork/Resources/apifiles/Python/gams"

In [2]:
model=pd.read_csv('/Users/hannahkamen/Downloads/windc_ind.csv')
output=pd.read_csv('/Users/hannahkamen/Downloads/y_rpt.csv')
cps=pd.read_stata('/Users/hannahkamen/Downloads/acs5yr_0610.dta')



In [364]:
lookup=pd.read_excel('/Users/hannahkamen/Downloads/windc_cps_industry_lookup.xlsx')
lookup_final=pd.read_excel('/Users/hannahkamen/Downloads/dropped_merge.xlsx')


lookup['NAICS']=lookup['NAICS'].astype(str).str.strip()
lookup_final['NAICS']=lookup_final['NAICS'].astype(str).str.strip()
lookup_final['cps_code']=lookup_final['cps_code'].astype(str).str.strip()

In [365]:
###read in model industry pairings in final data
gams_dta=pd.read_csv('/Users/hannahkamen/Downloads/windc_ld0_d.csv')


In [366]:
gams_dta.head()

,file,flx,int,scn,esubw_oth,sector,seniority,short,value
0,sm_test,IA,ppd,skl,0.249336,NaN,NaN,NaN,NaN
1,sm_test,IA,ppd,unskl,0.076501,NaN,NaN,NaN,NaN
2,sm_test,IA,res,skl,1.439216,NaN,NaN,NaN,NaN
3,sm_test,IA,res,unskl,0.441577,NaN,NaN,NaN,NaN
4,sm_test,IA,com,skl,0.856475,NaN,NaN,NaN,NaN


In [511]:
len(gams_dta_le0)

1366

In [367]:
state_lookup=pd.read_excel('/Users/hannahkamen/Downloads/state_lookup.xlsx')

In [368]:
state_lookup['state']=state_lookup['state'].str.strip()
state_lookup['abbrev']=state_lookup['abbrev'].str.strip()

In [369]:
###read in dropped 

In [370]:
cps_codes=pd.DataFrame(data={'cps_code':cps['indnaics'].str.strip().unique()})
windc_codes=pd.DataFrame(data={'windc_code':model['IOCode'].str.strip().unique()})

In [371]:
####match on first three symbols
cps_codes['cps_code3']=cps_codes['cps_code'].str[0:3]
lookup['NAICS3']=lookup['NAICS'].str[0:3]

merge3=lookup.merge(cps_codes, left_on='NAICS3',right_on='cps_code3',how='inner',indicator=True)

In [372]:
###append manual lookup to fill gaps
merge_f=merge3[['NAICS','cps_code']].append(lookup_final)

In [373]:
test=cps_codes.merge(merge_f,on='cps_code',how='outer',indicator=True)

In [374]:
cps_m=cps.merge(merge_f, left_on='indnaics', right_on='cps_code',how='left').merge(state_lookup, left_on='statefip',right_on='state', how='left')




In [375]:
cps_m=cps_m.rename(columns={'abbrev':'state_residence'})

In [376]:
cps_m=cps_m.merge(state_lookup, left_on='pwstate2', right_on='state',how='left')
cps_m=cps_m.rename(columns={'abbrev':'state_work'})

In [452]:
cps_m['educ_new']=np.where(cps_m['educ'].isin(['2 years of college','4 years of college','5+ years of college']),'skl','unskl')
                           
                           

In [453]:
cps_m['hh']=np.where(cps_m['incwage']<=25000,'hh1','')
cps_m['hh']=np.where(((cps_m['incwage']>25000) & (cps_m['incwage']<50000)),'hh2',cps_m['hh'])
cps_m['hh']=np.where(((cps_m['incwage']>=50000) & (cps_m['incwage']<75000)),'hh3',cps_m['hh'])
cps_m['hh']=np.where(((cps_m['incwage']>=75000) & (cps_m['incwage']<150000)),'hh4',cps_m['hh'])
cps_m['hh']=np.where((cps_m['incwage']>=150000),'hh5',cps_m['hh'])
 

#### get shares for consumer side, le0(r,q,h,sk)

In [454]:
gams_rqhs0=cps_m.reset_index()[['state_residence','state_work','hh','educ_new','incwage']].groupby(['state_residence','state_work','educ_new','hh'],as_index=False).agg({'incwage':sum})



In [455]:
gams_rqhs=gams_rqhs0.rename(columns={'state_residence':'r','state_work':'q','hh':'h','educ_new':'sk'})
gams_rqhs=gams_rqhs[['r','q','h','sk','incwage']]

In [456]:
midwest=['IA','OH','WI','NE','IL','MI','SD','ND','MN','MO','IN','KS']
south=['FL','MD','TN','WV','OK','KY','NC','VA','DE','GA','MS','TX','AL','LA','AR','SC','DC']
west=['AK','AZ','NM','HI','CA','WA','NV','OR','ID','UT','MT','WY','CO']
northeast=['VT','NH','CT','ME','MA','NY','NJ','PA','RI']

In [457]:
gams_rqhs['census_region_r']=np.where(gams_rqhs['r'].isin(midwest),'midwest','')
gams_rqhs['census_region_r']=np.where(gams_rqhs['r'].isin(south),'south',gams_rqhs['census_region_r'])
gams_rqhs['census_region_r']=np.where(gams_rqhs['r'].isin(west),'west',gams_rqhs['census_region_r'])
gams_rqhs['census_region_r']=np.where(gams_rqhs['r'].isin(northeast),'northeast',gams_rqhs['census_region_r'])


gams_rqhs['census_region_q']=np.where(gams_rqhs['q'].isin(midwest),'midwest','')
gams_rqhs['census_region_q']=np.where(gams_rqhs['q'].isin(south),'south',gams_rqhs['census_region_q'])
gams_rqhs['census_region_q']=np.where(gams_rqhs['q'].isin(west),'west',gams_rqhs['census_region_q'])
gams_rqhs['census_region_q']=np.where(gams_rqhs['q'].isin(northeast),'northeast',gams_rqhs['census_region_q'])

In [458]:
gams_rqhs_lm=gams_rqhs[(gams_rqhs['census_region_r']==gams_rqhs['census_region_q'])]

In [459]:
gams_rqhs_lm_gr=gams_rqhs_lm.groupby(['r','q','h'],as_index=False).agg({'incwage':sum})
gams_rqhs_lm_gr=gams_rqhs_lm_gr.rename(columns={'incwage':'incwage_total'})
gams_rqhs_lm1=gams_rqhs_lm.merge(gams_rqhs_lm_gr, on=['r','q','h'],how='left')
gams_rqhs_lm1['skl_shr']=gams_rqhs_lm1['incwage']/gams_rqhs_lm1['incwage_total']



In [460]:
gams_rqhs_lm1[(gams_rqhs_lm1['r']=='AK') &(gams_rqhs_lm1['q']=='AK')&(gams_rqhs_lm1['h']=='hh3')]

,r,q,h,sk,incwage,census_region_r,census_region_q,incwage_total,skl_shr
2,AK,AK,hh3,skl,131843519.0,west,west,251431093.0,0.524372
7,AK,AK,hh3,unskl,119587574.0,west,west,251431093.0,0.475628


In [515]:
gams_dta_le0.head()

,file,flx,int,scn,esubw_oth,sector,seniority,short,value
0,sm_test,IA,IA,hh1,skl,0.103923,NaN,NaN,NaN
1,sm_test,IA,IA,hh1,unskl,0.935311,NaN,NaN,NaN
2,sm_test,IA,IA,hh2,skl,0.667302,NaN,NaN,NaN
3,sm_test,IA,IA,hh2,unskl,3.781378,NaN,NaN,NaN
4,sm_test,IA,IA,hh3,skl,3.752220,NaN,NaN,NaN


In [516]:
list(gams_dta_le0)

['file',
 ' flx',
 ' int',
 ' scn',
 ' esubw_oth',
 ' sector',
 ' seniority',
 'short',
 ' value']

#### Deal with missing combinations in CPS data for r,q,h,sk

In [576]:
##prepare gams table for merge
gams_dta_le0=gams_dta_le0.rename(columns={' flx':'r',' int':'q',' scn':'h',' esubw_oth':'sk'})[['r','q','h','sk']]
##outer merge CPS data with gams table with indicator
merge_gams=gams_rqhs_lm1.merge(gams_dta_le0,on=['r','q','h','sk'],how='outer',indicator=True)
###get census region and assign average skilled and unskilled breakouts by census region for missing combinations
merge_gams['census_region']=np.where(merge_gams['r'].isin(midwest),'midwest','')
merge_gams['census_region']=np.where(merge_gams['r'].isin(south),'south',merge_gams['census_region'])
merge_gams['census_region']=np.where(merge_gams['r'].isin(west),'west',merge_gams['census_region'])
merge_gams['census_region']=np.where(merge_gams['r'].isin(northeast),'northeast',merge_gams['census_region'])

##get skilled unskilled breakouts by industry and census region

census_avg_tot0=merge_gams.groupby(['census_region','h'],as_index=False).agg({'incwage':sum})
census_avg_tot0=census_avg_tot0.rename(columns={'incwage':'incwage_tot'})
census_avg0=merge_gams.groupby(['census_region','h','sk'],as_index=False).agg({'incwage':sum})

census_avg0=census_avg0.merge(census_avg_tot0,on=['census_region','h'],how='inner')
census_avg0['census_share']=census_avg0['incwage']/census_avg0['incwage_tot']
del census_avg0['incwage']
del census_avg0['incwage_tot']

# merge census averages onto original merged dataframe
merge_gams_cs=merge_gams.merge(census_avg0,on=['census_region','h','sk'],how='inner')

##keep only inner values and values missing from CPS

final0=merge_gams_cs[merge_gams_cs['_merge'].isin(['right_only','both'])]

##get skilled unskilled breakouts by industry (ignore missing values for now)

final_tot0=final0.groupby(['r','q','h'],as_index=False).agg({'incwage':sum})
final_tot0=final_tot0.rename(columns={'incwage':'incwage_tot'})


final_shrs0=final0.merge(final_tot0,on=['r','q','h'],how='inner')
final_shrs0['skill_shr']=final_shrs0['incwage']/final_shrs0['incwage_tot']
del final_shrs0['incwage']
del final_shrs0['incwage_tot']

#create tage to define missing combinations in final dataset

final_shrs0['combo']=final_shrs0['r']+"_"+final_shrs0['q']+"_"+final_shrs0['h']

## get list of missing combionations using _merge indicator var

missing_combos0=final_shrs0[final_shrs0['_merge']=='right_only']['r']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['q']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['h']

###if region sector combination is not missing, use census region values, otherwise keep original share value
final_shrs0['skill_shr']=np.where(final_shrs0['combo'].isin(missing_combos0),final_shrs0['census_share'],final_shrs0['skill_shr'])
final_shrs0['skill_shr']=np.where(final_shrs0['combo']=='ME_NJ_hh1',final_shrs0['census_share'],final_shrs0['skill_shr'])
final_shrs0['skill_shr']=np.where(final_shrs0['combo'].isin(zeros),final_shrs0['census_share'],final_shrs0['skill_shr'])




###limit dataframe

final_shrs0=final_shrs0[['r','q','h','sk','skill_shr']]

#chksum to see that all proportions add to 1
check_sum0=final_shrs0.groupby(['r','q','h'],as_index=False).agg({'skill_shr':sum})
check_sum0=check_sum0[check_sum0['skill_shr']!=1]


#chksum to see that no zeros exist
check_sum10=final_shrs0.groupby(['r','q','h','sk'],as_index=False).agg({'skill_shr':sum})
check_sum10=check_sum10[check_sum10['skill_shr']==0]




In [577]:
check_sum0

,r,q,h,skill_shr


In [578]:
check_sum10

,r,q,h,sk,skill_shr


In [568]:
zeros=check_sum10['r']+"_"+check_sum10['q']+"_"+check_sum10['h']

In [579]:
len(final_shrs0)

6830

#### get tables by r,h,s

In [603]:
gams_dta_le0=pd.read_csv('/Users/hannahkamen/Downloads/windc_le0_d.csv')

In [604]:
##prepare gams table for merge
gams_dta_le0=gams_dta_le0.rename(columns={' flx':'r',' int':'q',' scn':'h',' esubw_oth':'sk',' sector':'pop'})[['r','q','h','sk','pop']]
##outer merge CPS data with gams table with indicator
merge_gams=gams_rqhs_lm1.merge(gams_dta_le0,on=['r','q','h','sk'],how='outer',indicator=True)
###get census region and assign average skilled and unskilled breakouts by census region for missing combinations
merge_gams['census_region']=np.where(merge_gams['r'].isin(midwest),'midwest','')
merge_gams['census_region']=np.where(merge_gams['r'].isin(south),'south',merge_gams['census_region'])
merge_gams['census_region']=np.where(merge_gams['r'].isin(west),'west',merge_gams['census_region'])
merge_gams['census_region']=np.where(merge_gams['r'].isin(northeast),'northeast',merge_gams['census_region'])

##get skilled unskilled breakouts by industry and census region

census_avg_tot0=merge_gams.groupby(['census_region','h'],as_index=False).agg({'incwage':sum})
census_avg_tot0=census_avg_tot0.rename(columns={'incwage':'incwage_tot'})
census_avg0=merge_gams.groupby(['census_region','h','sk'],as_index=False).agg({'incwage':sum})

census_avg0=census_avg0.merge(census_avg_tot0,on=['census_region','h'],how='inner')
census_avg0['census_share']=census_avg0['incwage']/census_avg0['incwage_tot']
del census_avg0['incwage']
del census_avg0['incwage_tot']

# merge census averages onto original merged dataframe
merge_gams_cs=merge_gams.merge(census_avg0,on=['census_region','h','sk'],how='inner')

##keep only inner values and values missing from CPS

final0=merge_gams_cs[merge_gams_cs['_merge'].isin(['right_only','both'])]

##get skilled unskilled breakouts by industry (ignore missing values for now)

final_tot0=final0.groupby(['r','q','h'],as_index=False).agg({'incwage':sum})
final_tot0=final_tot0.rename(columns={'incwage':'incwage_tot'})


final_shrs0=final0.merge(final_tot0,on=['r','q','h'],how='inner')
final_shrs0['skill_shr']=final_shrs0['incwage']/final_shrs0['incwage_tot']
del final_shrs0['incwage']
del final_shrs0['incwage_tot']

#create tage to define missing combinations in final dataset

final_shrs0['combo']=final_shrs0['r']+"_"+final_shrs0['q']+"_"+final_shrs0['h']

## get list of missing combionations using _merge indicator var

missing_combos0=final_shrs0[final_shrs0['_merge']=='right_only']['r']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['q']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['h']

###if region sector combination is not missing, use census region values, otherwise keep original share value
final_shrs0['skill_shr']=np.where(final_shrs0['combo'].isin(missing_combos0),final_shrs0['census_share'],final_shrs0['skill_shr'])
final_shrs0['skill_shr']=np.where(final_shrs0['combo']=='ME_NJ_hh1',final_shrs0['census_share'],final_shrs0['skill_shr'])
final_shrs0['skill_shr']=np.where(final_shrs0['combo'].isin(zeros),final_shrs0['census_share'],final_shrs0['skill_shr'])




###limit dataframe

final_shrs0=final_shrs0[['r','q','h','sk','skill_shr','pop']]

#chksum to see that all proportions add to 1
check_sum0=final_shrs0.groupby(['r','q','h'],as_index=False).agg({'skill_shr':sum})
check_sum0=check_sum0[check_sum0['skill_shr']!=1]


#chksum to see that no zeros exist
check_sum10=final_shrs0.groupby(['r','q','h','sk'],as_index=False).agg({'skill_shr':sum})
check_sum10=check_sum10[check_sum10['skill_shr']==0]




In [633]:
final_shrs0

,r,q,h,sk,skill_shr
0,AK,AK,hh1,skl,0.210555
1,AK,AK,hh1,unskl,0.789445
2,AK,AZ,hh1,skl,0.264425
3,AK,AZ,hh1,unskl,0.735575
4,AK,CA,hh1,skl,0.491478
...,...,...,...,...,...
6825,ND,MO,hh5,unskl,0.121915
6826,ND,IN,hh5,skl,0.878085
6827,ND,IN,hh5,unskl,0.121915
6828,ND,KS,hh5,skl,0.878085


In [612]:
#group to sum pop

final_shrs0_gr=final_shrs0.groupby(['r','q','h'],as_index=False).agg({'pop':sum})
del final_shrs0['pop']
#final_shrs0_sum=final_shrs0.groupby(['r','h','sk'])

In [627]:
final_shrs0_popm=final_shrs0.merge(final_shrs0_gr,on=['r','q','h'],how='inner')
final_shrs0_popm['pop_skl']=final_shrs0_popm['skill_shr']*final_shrs0_popm['pop']

###get pop shares by r,h,sk

final_shrs0_popm_gr=final_shrs0_popm.groupby(['r','h','sk'],as_index=False).agg({'pop_skl':sum})

##get total pop

final_shrs0_popm_gr_tot=final_shrs0_popm_gr.groupby(['r','h'],as_index=False).agg({'pop_skl':sum})
final_shrs0_popm_gr_tot=final_shrs0_popm_gr_tot.rename(columns={'pop_skl':'pop_skl_tot'})

##merge on to r,h,sk dataset

final_shrs0_popm_gr=final_shrs0_popm_gr.merge(final_shrs0_popm_gr_tot, on=['r','h'],how='inner')
final_shrs0_popm_gr['skill_shr']=final_shrs0_popm_gr['pop_skl']/final_shrs0_popm_gr['pop_skl_tot']

del final_shrs0_popm_gr['pop_skl']
del final_shrs0_popm_gr['pop_skl_tot']

check_sum0=final_shrs0_popm_gr.groupby(['r','h'],as_index=False).agg({'skill_shr':sum})
check_sum0[check_sum0['skill_shr']!=1]
len(check_sum0[check_sum0['skill_shr']!=1])

check_sum10=final_shrs0_popm_gr.groupby(['r','h','sk'],as_index=False).agg({'skill_shr':sum})
check_sum10=check_sum10[check_sum10['skill_shr']==0]

In [631]:
final_shrs0_popm_gr

,r,h,sk,skill_shr
0,AK,hh1,skl,0.228554
1,AK,hh1,unskl,0.771446
2,AK,hh2,skl,0.383052
3,AK,hh2,unskl,0.616948
4,AK,hh3,skl,0.546785
...,...,...,...,...
505,WY,hh3,unskl,0.455958
506,WY,hh4,skl,0.603558
507,WY,hh4,unskl,0.396442
508,WY,hh5,skl,0.729101


In [632]:
final_shrs0

,r,q,h,sk,skill_shr
0,AK,AK,hh1,skl,0.210555
1,AK,AK,hh1,unskl,0.789445
2,AK,AZ,hh1,skl,0.264425
3,AK,AZ,hh1,unskl,0.735575
4,AK,CA,hh1,skl,0.491478
...,...,...,...,...,...
6825,ND,MO,hh5,unskl,0.121915
6826,ND,IN,hh5,skl,0.878085
6827,ND,IN,hh5,unskl,0.121915
6828,ND,KS,hh5,skl,0.878085


In [629]:
len(final_shrs0_popm_gr)

510

In [628]:
check_sum10

,r,h,sk,skill_shr


In [625]:
check_sum0[check_sum0['skill_shr']!=1]['skill_shr'].sum()

33.99999999999999

In [634]:
#final_shrs0_popm_gr.to_csv('/Users/hannahkamen/Downloads/le0_s_shr.csv',index=False)

final_shrs0.to_csv('/Users/hannahkamen/Downloads/le0_shr.csv',index=False)
#final_shrs.to_csv('/Users/hannahkamen/Downloads/ld0_shr.csv',index=False)

,r,q,h,sk,skill_shr
0,AK,AK,hh1,skl,0.210555
1,AK,AK,hh1,unskl,0.789445
2,AK,AZ,hh1,skl,0.264425
3,AK,AZ,hh1,unskl,0.735575
4,AK,CA,hh1,skl,0.491478
...,...,...,...,...,...
6825,ND,MO,hh5,unskl,0.121915
6826,ND,IN,hh5,skl,0.878085
6827,ND,IN,hh5,unskl,0.121915
6828,ND,KS,hh5,skl,0.878085


In [ ]:
final_shrs1.groupby()

In [580]:
len(gams_dta_le0)

6830

#### get shares for producer side  ld0(r,s,sk)


In [548]:
# group on sector
gams_rssk0=cps_m.reset_index()[['state_work','NAICS','educ_new','incwage']].groupby(['state_work','NAICS','educ_new'],as_index=False).agg({'incwage':sum})
#get rid of puerto rico and "bad" sectors



In [549]:
##rename to gams indices
gams_rssk=gams_rssk0.rename(columns={'state_work':'r','NAICS':'s','educ_new':'sk'})
gams_rssk=gams_rssk[['r','s','sk','incwage']]

In [550]:
model['gams.Commodities/Industries'].unique()

array(['agr', 'fof', 'oil', 'min', 'smn', 'uti', 'con', 'wpd', 'nmp',
       'pmt', 'fmt', 'mch', 'cep', 'eec', 'mot', 'ote', 'fpd', 'mmf',
       'fbp', 'tex', 'alt', 'ppd', 'pri', 'pet', 'che', 'pla', 'wht',
       'mvt', 'fbt', 'gmt', 'ott', 'air', 'trn', 'wtt', 'trk', 'grd',
       'pip', 'otr', 'wrh', 'pub', 'mov', 'brd', 'dat', 'bnk', 'sec',
       'ins', 'fin', 'hou', 'ore', 'rnt', 'leg', 'com', 'tsv', 'man',
       'adm', 'wst', 'edu', 'amb', 'hos', 'nrs', 'soc', 'art', 'rec',
       'amd', 'res', 'osv', 'fdd', 'fnd', 'fen', 'slg', 'sle', 'Output',
       'imports', 'ciffob', 'BasicSupply', 'Margins', 'TrnCost', 'TrdTrn',
       'Duties', 'Tax', 'Subsidies', 'TaxLesSubsidies', 'Supply'],
      dtype=object)

In [551]:
model['gams.Commodities/Industries'].unique()

non_s=['Output',
       'imports', 'ciffob', 'BasicSupply', 'Margins', 'TrnCost', 'TrdTrn',
       'Duties', 'Tax', 'Subsidies', 'TaxLesSubsidies', 'Supply']

In [552]:
model_gr=model.groupby(['IOCode','gams.IOCode'],as_index=False).sum()
model_gr=model_gr[['IOCode','gams.IOCode']]

In [553]:
gams_rssk_m=gams_rssk.merge(model_gr, left_on='s',right_on='IOCode',how='left')
gams_rssk_m=gams_rssk_m[gams_rssk_m['gams.IOCode']!='use']
gams_rssk_m=gams_rssk_m[gams_rssk_m['r']!='PR']
del gams_rssk_m['s']
del gams_rssk_m['IOCode']
gams_rssk_m=gams_rssk_m.rename(columns={'gams.IOCode':'s'})
gams_rssk_m=gams_rssk_m[['r','s','sk','incwage']]

In [635]:
gams_rssk_m

,r,s,sk,incwage
0,AK,agr,skl,260334.0
1,AK,agr,unskl,272796.0
2,AK,fof,skl,3821554.0
3,AK,fof,unskl,6436523.0
4,AK,oil,skl,6499426.0
...,...,...,...,...
7336,WY,slg,unskl,16264500.0
7337,WY,hou,skl,2924531.0
7338,WY,hou,unskl,2124463.0
7339,WY,ore,skl,3044200.0


#### figure out which combinations are not in CPS but in GAMS tables

In [558]:
##prepare gams table for merge
gams_dta=gams_dta.rename(columns={' flx':'r',' int':'s',' scn':'sk'})[['r','s','sk']]
##outer merge CPS data with gams table with indicator
merge_sector=gams_rssk_m.merge(gams_dta,on=['r','s','sk'],how='outer',indicator=True)

###get census region and assign average skilled and unskilled breakouts by census region for missing combinations
merge_sector['census_region']=np.where(merge_sector['r'].isin(midwest),'midwest','')
merge_sector['census_region']=np.where(merge_sector['r'].isin(south),'south',merge_sector['census_region'])
merge_sector['census_region']=np.where(merge_sector['r'].isin(west),'west',merge_sector['census_region'])
merge_sector['census_region']=np.where(merge_sector['r'].isin(northeast),'northeast',merge_sector['census_region'])

##get skilled unskilled breakouts by industry and census region

census_avg_tot=merge_sector.groupby(['census_region','s'],as_index=False).agg({'incwage':sum})
census_avg_tot=census_avg_tot.rename(columns={'incwage':'incwage_tot'})
census_avg=merge_sector.groupby(['census_region','s','sk'],as_index=False).agg({'incwage':sum})

census_avg=census_avg.merge(census_avg_tot,on=['census_region','s'],how='inner')
census_avg['census_share']=census_avg['incwage']/census_avg['incwage_tot']
del census_avg['incwage']
del census_avg['incwage_tot']

# merge census averages onto original merged dataframe
merge_sector_cs=merge_sector.merge(census_avg,on=['census_region','s','sk'],how='inner')

##keep only inner values and values missing from CPS

final=merge_sector_cs[merge_sector_cs['_merge'].isin(['right_only','both'])]

##get skilled unskilled breakouts by industry (ignore missing values for now)

final_tot=final.groupby(['r','s'],as_index=False).agg({'incwage':sum})
final_tot=final_tot.rename(columns={'incwage':'incwage_tot'})


final_shrs=final.merge(final_tot,on=['r','s'],how='inner')
final_shrs['skill_shr']=final_shrs['incwage']/final_shrs['incwage_tot']
del final_shrs['incwage']
del final_shrs['incwage_tot']

#create tage to define missing combinations in final dataset

final_shrs['combo']=final_shrs['r']+"_"+final_shrs['s']

## get list of missing combionations using _merge indicator var

missing_combos=final_shrs[final_shrs['_merge']=='right_only']['r']+"_"+final_shrs[final_shrs['_merge']=='right_only']['s']

###if region sector combination is not missing, use census region values, otherwise keep original share value
final_shrs['skill_shr']=np.where(final_shrs['combo'].isin(missing_combos),final_shrs['census_share'],final_shrs['skill_shr'])
final_shrs['skill_shr']=np.where(final_shrs['combo']=='AK_alt',final_shrs['census_share'],final_shrs['skill_shr'])

###limit dataframe

final_shrs=final_shrs[['r','s','sk','skill_shr']]


#chksum to see that all proportions add to 1
check_sum=final_shrs.groupby(['r','s'],as_index=False).agg({'skill_shr':sum})
check_sum[check_sum['skill_shr']!=1]

#chksum to see t no zeros exist
check_sum1=final_shrs.groupby(['r','s','sk'],as_index=False).agg({'skill_shr':sum})
check_sum1[check_sum1['skill_shr']==0]



,r,s,sk,skill_shr


In [ ]:
final_shrs.groupby(['r','sk',])

In [582]:
check_sum[check_sum['skill_shr']!=1]

,r,s,skill_shr


In [583]:
check_sum1[check_sum1['skill_shr']==1]

,r,s,sk,skill_shr


In [584]:
len(gams_dta)

7202

In [585]:
len(final_shrs)

7202

In [421]:
# in_table=gams_rssk_m['gams.IOCode'].unique()
# in_model_dta=gams_dta[' int'].unique()
# [x for x in in_table if x not in in_model_dta]
# test1=gams_rssk['s'].unique()
# test2=model[~model['gams.Commodities/Industries'].isin(non_s)]['gams.Commodities/Industries'].unique()
# test3=[x for x in test2 if x not in test1 ]
# print(len(midwest))
# print(len(south))
# print(len(west))
# print(len(northeast))

In [17]:
model1=model[~model['gams.Commodities/Industries'].isin(['Output',
       'imports', 'ciffob', 'BasicSupply', 'Margins', 'TrnCost', 'TrdTrn',
       'Duties', 'Tax', 'Subsidies', 'TaxLesSubsidies', 'Supply'])]

In [18]:
len(model1['gams.Commodities/Industries'].unique())

71